In [1]:
import os
import datetime
import lxml.etree as et
import csv
pmcdir = 'pmc_articles'
from plos_corpus import (corpusdir, get_article_pubdate, check_if_uncorrected_proof, listdir_nohidden,
                         get_article_xml, file_to_doi, doi_to_file)

from samples.corpus_analysis import (get_plos_article_type, get_article_dtd, get_random_list_of_dois, 
                                     get_related_retraction_article, check_article_type, get_plos_journal,
                                     get_article_title, parse_article_date, get_corpus_metadata,
                                     get_article_abstract, corpus_metadata_to_csv)


In [15]:
article_files = [doi_to_file(doi) for doi in get_random_list_of_dois(count=1000)]


In [17]:
corpus_metadata = get_corpus_metadata(article_files)

100% (1000 of 1000) |#####################| Elapsed Time: 0:00:26 Time: 0:00:26


In [16]:
for article in article_files[30:40]:
    abstract = get_article_abstract(article)
    print(abstract)


Tumorigenesis requires the re-organization of metabolism to support malignant proliferation. We examine how the altered metabolism of cancer cells is reflected in the rewiring of co-expression patterns among metabolic genes. Focusing on breast and clear-cell kidney tumors, we report the existence of key metabolic genes which act as hubs of differential co-expression, showing significantly different co-regulation patterns between normal and tumor states. We compare our findings to those from classical differential expression analysis, and counterintuitively observe that the extent of a gene's differential co-expression only weakly correlates with its differential expression, suggesting that the two measures probe different features of metabolism. Focusing on this discrepancy, we use changes in co-expression patterns to highlight the apparent loss of regulation by the transcription factor HNF4A in clear cell renal cell carcinoma, despite no differential expression of HNF4A. Finally, we 

In [ ]:
def get_article_abstract(article_file):
    """
    For an individual article in the PLOS corpus, create a tuple of a set of metadata fields sbout that corpus.
    Make it small, medium, or large depending on number of fields desired.
    :param article_file: individual local PLOS XML article
    :return: plain-text string of content in abstract
    """
    abstract = get_article_xml(article_file, tag_path_elements=["/",
                                                                "article",
                                                                "front",
                                                                "article-meta",
                                                                "abstract"])
    try:
        abstract_text = et.tostring(abstract[0], encoding='unicode', method='text')
    except IndexError:
        if check_article_type(article_file) == 'research-article' and \
          get_plos_article_type(article_file) == 'Research Article':
            print(check_article_type(article_file), article_file)

        abstract_text = ''
    # clean up text: rem white space, new line marks
    abstract_text.replace('  ', '').strip()
    if '\n' in abstract_text:
        print(abstract_text.decode('utf-8'))

    return abstract_text

In [18]:
corpus_metadata_to_csv(corpus_metadata=corpus_metadata)

In [ ]:
abstract = get_article_abstract()
print(abstract)

In [ ]:
abstract = b"".decode("utf-8") 

In [10]:
def get_article_metadata(article_file, size='small'):
    """
    For an individual article in the PLOS corpus, create a tuple of a set of metadata fields sbout that corpus.
    Make it small, medium, or large depending on number of fields desired.
    :param article_file: individual local PLOS XML article
    :param size: small, medium or large, aka how many fields to return for each article
    :return: tuple of metadata fields
    """
    doi = file_to_doi(article_file)
    filename = os.path.basename(article_file).rstrip('.xml')
    title = get_article_title(article_file)
    journal = get_plos_journal(article_file)
    jats_article_type = check_article_type(article_file)
    plos_article_type = get_plos_article_type(article_file)
    dtd_version = get_article_dtd(article_file)
    dates = get_article_dates(article_file, string=True)
    (pubdate, collection, received, accepted) = ('','','','')
    pubdate = dates['epub']
    abstract = get_article_abstract(article_file)
    try:
        collection = dates['collection']
    except KeyError:
        pass
    try:
        received = dates['received']
    except KeyError:
        pass
    try:
        accepted = dates['accepted']
    except KeyError:
        pass
    metadata = [doi, filename, title, journal, jats_article_type, plos_article_type, dtd_version, pubdate,
                received, accepted, collection, abstract]
    metadata = tuple(metadata)
    if len(metadata) == 12:
        return metadata
    else:
        print('Error in {}: {} items'.format(article_file, len(article_file)))
        return False

In [ ]:
get_article_abstract('10.1371/journal.pone.0118395')

In [4]:
one_list = [article for article in listdir_nohidden(corpusdir) if 'pone' in article]
print(len(one_list))
sample_list = one_list[0:1000]

def assemble_diabetes_corpus(article_list):
    """
    Find all PLOS ONE articles that say something about diabetes or technology in the abstract.
    """
    diabetes_article_list = [article for article in article_list if all(c in get_article_abstract(article).lower() for c in ('diabet', 'techno'))]
    return diabetes_article_list

187985


In [2]:
def assemble_diabetes_corpus(article_list):
    """
    Find all PLOS ONE articles that say something about diabetes or technology in the abstract.
    """
    diabetes_article_list = [article for article in article_list if 'diabet' in get_article_abstract(article).lower()]
    return diabetes_article_list

In [5]:
diabetes_article_list = assemble_diabetes_corpus(one_list)

research-article allofplos_xml/journal.pone.0150341.xml
research-article allofplos_xml/journal.pone.0160248.xml
research-article allofplos_xml/journal.pone.0163841.xml
research-article allofplos_xml/journal.pone.0173427.xml
research-article allofplos_xml/journal.pone.0174259.xml
research-article allofplos_xml/journal.pone.0184204.xml


In [8]:
diabetes_metadata = get_corpus_metadata(article_list=diabetes_article_list)

100% (72 of 72) |#########################| Elapsed Time: 0:00:02 Time: 0:00:02


In [14]:
corpus_metadata_to_csv(diabetes_metadata)

In [ ]:
print(diabetes_metadata)

In [6]:
with open('diabetes.txt', 'w') as f:
    for article in diabetes_article_list:
        f.write("%s\n" % '@@@')
        f.write("%s\n" % file_to_doi(article))
        f.write("%s\n" % '$$$')
        f.write("%s\n" % get_article_title(article))
        f.write("%s\n" % '###')
        f.write("%s\n" % get_article_abstract(article))

In [ ]:
diabetes_abstract_list = [get_article_abstract(article) for article in diabetes_article_list]

In [11]:
diabetes_metadata[0]

('10.1371/journal.pone.0004907',
 'journal.pone.0004907',
 'De-Novo Identification of PPARγ/RXR Binding Sites and Direct Targets during Adipogenesis',
 'PLOS ONE',
 'research-article',
 'Research Article',
 '3.0',
 '2009-03-20',
 '2008-10-28',
 '2009-01-28',
 '2009-01-01',
 '\nBackground\nThe pathophysiology of obesity and type 2 diabetes mellitus is associated with abnormalities in endocrine signaling in adipose tissue and one of the key signaling affectors operative in these disorders is the nuclear hormone transcription factor peroxisome proliferator-activated receptor-γ (PPARγ). PPARγ has pleiotropic functions affecting a wide range of fundamental biological processes including the regulation of genes that modulate insulin sensitivity, adipocyte differentiation, inflammation and atherosclerosis. To date, only a limited number of direct targets for PPARγ have been identified through research using the well established pre-adipogenic cell line, 3T3-L1. In order to obtain a genome-wid

In [13]:
def corpus_metadata_to_csv(corpus_metadata=None):
    """
    Convert list of tuples from get_article_metadata to csv
    :param corpus_metadata: the list of tuples, defaults to creating from corpusdir
    :return: None
    """
    if corpus_metadata is None:
        corpus_metadata = get_corpus_metadata()
    with open('allofplos_metadata.csv', 'w') as out:
        csv_out = csv.writer(out)
        csv_out.writerow(['doi', 'filename', 'title', 'journal', 'jats_article_type', 'plos_article_type',
                          'dtd_version', 'pubdate', 'received', 'accepted', 'collection', 'abstract'])
        for row in corpus_metadata:
            csv_out.writerow(row)

In [64]:
def get_article_counts(article_file):
    """
    For a single article, return a dictionary of the several counts functions that are available
    (figures: fig-count, pages: page-count, tables: table-count)
    :return: counts dictionary
    """
    counts = {}

    tag_path = ["/",
                "article",
                "front",
                "article-meta",
                "counts"]
    raw_xml = get_article_xml(article_file=article_file,
                              tag_path_elements=tag_path)
    for element in raw_xml:
        for count_item in element:
            count = count_item.get('count')
            count_type = count_item.tag
            counts[count_type] = count
    if len(counts) > 3:
        print(counts)
    return counts

In [70]:
article_list = get_random_list_of_dois(count=10000)

In [72]:
for article in article_list:
    print(get_article_counts(article))

{'page-count': '8'}
{}
{'fig-count': '3', 'table-count': '5', 'page-count': '15'}
{'fig-count': '0', 'table-count': '0', 'page-count': '1'}
{'fig-count': '2', 'table-count': '4', 'page-count': '12'}
{'page-count': '14'}
{}
{'page-count': '9'}
{'page-count': '8'}
{'fig-count': '7', 'table-count': '4', 'page-count': '25'}
{'fig-count': '5', 'table-count': '6', 'page-count': '15'}
{'page-count': '9'}
{'page-count': '12'}
{'fig-count': '6', 'table-count': '1', 'page-count': '14'}
{'page-count': '5'}
{'page-count': '11'}
{'page-count': '9'}
{'page-count': '9'}
{'page-count': '9'}
{'page-count': '12'}
{'page-count': '12'}
{'fig-count': '11', 'table-count': '0', 'page-count': '33'}
{'page-count': '8'}
{'fig-count': '6', 'table-count': '5', 'page-count': '16'}
{'page-count': '7'}
{'page-count': '7'}
{'page-count': '8'}
{'page-count': '6'}
{'fig-count': '4', 'table-count': '3', 'page-count': '14'}
{'fig-count': '21', 'table-count': '3', 'page-count': '29'}
{'fig-count': '7', 'table-count': '0',

{'fig-count': '18', 'table-count': '9', 'page-count': '31'}
{'page-count': '15'}
{'page-count': '9'}
{'page-count': '4'}
{'fig-count': '1', 'table-count': '3', 'page-count': '14'}
{'page-count': '10'}
{'page-count': '15'}
{'page-count': '12'}
{}
{}
{'fig-count': '9', 'table-count': '0', 'page-count': '31'}
{'page-count': '7'}
{'fig-count': '5', 'table-count': '0', 'page-count': '18'}
{'page-count': '1'}
{'page-count': '9'}
{}
{'page-count': '6'}
{'fig-count': '3', 'table-count': '0', 'page-count': '14'}
{'page-count': '9'}
{'page-count': '23'}
{'page-count': '18'}
{'fig-count': '2', 'table-count': '1', 'page-count': '10'}
{'page-count': '9'}
{'page-count': '13'}
{'fig-count': '2', 'table-count': '0', 'page-count': '9'}
{'page-count': '17'}
{'page-count': '7'}
{'fig-count': '1', 'table-count': '4', 'page-count': '9'}
{'fig-count': '3', 'table-count': '0', 'page-count': '5'}
{'fig-count': '1', 'table-count': '0', 'page-count': '7'}
{'page-count': '5'}
{'page-count': '9'}
{'fig-count': '7

{'fig-count': '8', 'table-count': '2', 'page-count': '16'}
{'page-count': '2'}
{'page-count': '8'}
{'fig-count': '2', 'table-count': '3', 'page-count': '17'}
{'page-count': '11'}
{'page-count': '8'}
{'page-count': '11'}
{'page-count': '13'}
{'fig-count': '10', 'table-count': '3', 'page-count': '29'}
{'fig-count': '8', 'table-count': '6', 'page-count': '31'}
{'page-count': '5'}
{'fig-count': '9', 'table-count': '0', 'page-count': '18'}
{'fig-count': '2', 'table-count': '2', 'page-count': '12'}
{'page-count': '12'}
{'fig-count': '7', 'table-count': '0', 'page-count': '20'}
{'page-count': '7'}
{'page-count': '8'}
{'page-count': '7'}
{'page-count': '6'}
{'page-count': '12'}
{'fig-count': '5', 'table-count': '4', 'page-count': '18'}
{'fig-count': '3', 'table-count': '1', 'page-count': '11'}
{'page-count': '18'}
{'page-count': '7'}
{'page-count': '11'}
{'fig-count': '7', 'table-count': '0', 'page-count': '19'}
{'page-count': '9'}
{'fig-count': '1', 'table-count': '5', 'page-count': '14'}
{'f

{'page-count': '24'}
{'page-count': '10'}
{'fig-count': '15', 'table-count': '0', 'page-count': '37'}
{'fig-count': '5', 'table-count': '3', 'page-count': '14'}
{'fig-count': '0', 'table-count': '0', 'page-count': '1'}
{'page-count': '13'}
{'fig-count': '1', 'table-count': '4', 'page-count': '12'}
{'page-count': '12'}
{'fig-count': '4', 'table-count': '0', 'page-count': '12'}
{'fig-count': '5', 'table-count': '3', 'page-count': '16'}
{'page-count': '11'}
{'fig-count': '7', 'table-count': '0', 'page-count': '18'}
{'fig-count': '9', 'table-count': '1', 'page-count': '26'}
{'fig-count': '5', 'table-count': '4', 'page-count': '19'}
{'page-count': '10'}
{'page-count': '8'}
{'page-count': '9'}
{'fig-count': '6', 'table-count': '0', 'page-count': '22'}
{'page-count': '11'}
{'page-count': '6'}
{'fig-count': '6', 'table-count': '0', 'page-count': '16'}
{'page-count': '9'}
{}
{'page-count': '11'}
{'fig-count': '4', 'table-count': '1', 'page-count': '12'}
{'fig-count': '9', 'table-count': '0', 'p

{'fig-count': '6', 'table-count': '4', 'page-count': '13'}
{'page-count': '12'}
{'page-count': '7'}
{'page-count': '8'}
{'fig-count': '7', 'table-count': '6', 'page-count': '17'}
{'page-count': '14'}
{'page-count': '5'}
{'page-count': '21'}
{'page-count': '14'}
{}
{'page-count': '12'}
{'fig-count': '9', 'table-count': '0', 'page-count': '20'}
{'fig-count': '11', 'table-count': '2', 'page-count': '23'}
{'page-count': '16'}
{'page-count': '12'}
{'page-count': '9'}
{}
{'fig-count': '10', 'table-count': '7', 'page-count': '32'}
{'fig-count': '0', 'table-count': '0', 'page-count': '1'}
{'page-count': '15'}
{'fig-count': '7', 'table-count': '0', 'page-count': '25'}
{'page-count': '25'}
{'page-count': '9'}
{'page-count': '13'}
{'page-count': '15'}
{'page-count': '15'}
{'page-count': '13'}
{'fig-count': '6', 'table-count': '2', 'page-count': '14'}
{'page-count': '7'}
{'fig-count': '7', 'table-count': '2', 'page-count': '25'}
{'page-count': '7'}
{'fig-count': '3', 'table-count': '1', 'page-coun

...

{'page-count': '16'}
{'fig-count': '8', 'table-count': '0', 'page-count': '30'}
{'page-count': '10'}
{'page-count': '4'}
{'page-count': '13'}
{'page-count': '8'}
{'fig-count': '3', 'table-count': '4', 'page-count': '17'}
{'page-count': '10'}
{'page-count': '6'}
{'page-count': '12'}
{'page-count': '7'}
{'page-count': '7'}
{}
{'page-count': '7'}
{'page-count': '15'}
{'page-count': '18'}
{'page-count': '8'}
{'page-count': '9'}
{'page-count': '8'}
{'fig-count': '8', 'table-count': '14', 'page-count': '30'}
{'page-count': '5'}
{'fig-count': '1', 'table-count': '0', 'page-count': '8'}
{'page-count': '13'}
{'page-count': '11'}
{'page-count': '12'}
{}
{'fig-count': '6', 'table-count': '1', 'page-count': '17'}
{'page-count': '11'}
{'page-count': '15'}
{'fig-count': '10', 'table-count': '1', 'page-count': '24'}
{'page-count': '10'}
{'page-count': '9'}
{}
{'page-count': '13'}
{'page-count': '17'}
{'fig-count': '7', 'table-count': '1', 'page-count': '29'}
{'fig-count': '1', 'table-count': '5', 'pa

{'fig-count': '2', 'table-count': '1', 'page-count': '11'}
{'fig-count': '5', 'table-count': '1', 'page-count': '12'}
{'page-count': '10'}
{'fig-count': '4', 'table-count': '2', 'page-count': '19'}
{'fig-count': '1', 'table-count': '0', 'page-count': '1'}
{'page-count': '8'}
{'page-count': '15'}
{'page-count': '11'}
{'page-count': '18'}
{'page-count': '12'}
{'page-count': '7'}
{'page-count': '17'}
{'fig-count': '1', 'table-count': '3', 'page-count': '15'}
{'fig-count': '17', 'table-count': '8', 'page-count': '27'}
{'fig-count': '6', 'table-count': '1', 'page-count': '19'}
{'fig-count': '0', 'table-count': '3', 'page-count': '16'}
{'fig-count': '0', 'table-count': '3', 'page-count': '10'}
{'fig-count': '7', 'table-count': '2', 'page-count': '18'}
{'page-count': '9'}
{'fig-count': '4', 'table-count': '2', 'page-count': '16'}
{'page-count': '12'}
{'page-count': '8'}
{'page-count': '12'}
{'fig-count': '0', 'table-count': '4', 'page-count': '11'}
{'page-count': '7'}
{'page-count': '12'}
{'p

{'page-count': '9'}
{'page-count': '11'}
{'fig-count': '5', 'table-count': '1', 'page-count': '12'}
{'page-count': '13'}
{'page-count': '5'}
{'fig-count': '5', 'table-count': '0', 'page-count': '23'}
{'page-count': '7'}
{'page-count': '2'}
{'page-count': '12'}
{'fig-count': '4', 'table-count': '2', 'page-count': '17'}
{'fig-count': '6', 'table-count': '0', 'page-count': '15'}
{'fig-count': '7', 'table-count': '0', 'page-count': '19'}
{'page-count': '8'}
{'page-count': '6'}
{'fig-count': '2', 'table-count': '4', 'page-count': '13'}
{'page-count': '9'}
{'fig-count': '2', 'table-count': '3', 'page-count': '16'}
{'page-count': '11'}
{'page-count': '8'}
{'fig-count': '4', 'table-count': '2', 'page-count': '18'}
{'page-count': '15'}
{'fig-count': '4', 'table-count': '2', 'page-count': '15'}
{}
{'fig-count': '4', 'table-count': '1', 'page-count': '13'}
{'fig-count': '11', 'table-count': '3', 'page-count': '20'}
{'page-count': '12'}
{'page-count': '9'}
{}
{'page-count': '10'}
{'page-count': '9

{'page-count': '10'}
{'page-count': '13'}
{'page-count': '10'}
{'fig-count': '6', 'table-count': '0', 'page-count': '17'}
{'page-count': '9'}
{'page-count': '11'}
{'page-count': '9'}
{'page-count': '4'}
{'page-count': '36'}
{}
{'page-count': '14'}
{'fig-count': '2', 'table-count': '5', 'page-count': '15'}
{'fig-count': '1', 'table-count': '3', 'page-count': '9'}
{'page-count': '9'}
{'page-count': '8'}
{}
{}
{'page-count': '14'}
{'fig-count': '1', 'table-count': '0', 'page-count': '2'}
{'fig-count': '7', 'table-count': '0', 'page-count': '17'}
{'page-count': '8'}
{'page-count': '8'}
{'page-count': '10'}
{'fig-count': '10', 'table-count': '0', 'page-count': '27'}
{'fig-count': '1', 'table-count': '5', 'page-count': '15'}
{}
{'page-count': '16'}
{'page-count': '11'}
{'page-count': '8'}
{'page-count': '7'}
{'fig-count': '4', 'table-count': '0', 'page-count': '13'}
{'page-count': '18'}
{'page-count': '10'}
{'fig-count': '0', 'table-count': '7', 'page-count': '21'}
{'page-count': '11'}
{'pag

{'page-count': '8'}
{'page-count': '12'}
{'page-count': '8'}
{'fig-count': '5', 'table-count': '0', 'page-count': '17'}
{'page-count': '7'}
{'page-count': '9'}
{'page-count': '12'}
{'fig-count': '7', 'table-count': '5', 'page-count': '18'}
{'fig-count': '0', 'table-count': '4', 'page-count': '12'}
{'page-count': '11'}
{'page-count': '9'}
{'page-count': '7'}
{'page-count': '15'}
{'page-count': '11'}
{'page-count': '13'}
{'page-count': '20'}
{'page-count': '10'}
{'page-count': '1'}
{'page-count': '10'}
{'fig-count': '1', 'table-count': '2', 'page-count': '9'}
{'page-count': '9'}
{'fig-count': '6', 'table-count': '1', 'page-count': '18'}
{'fig-count': '1', 'table-count': '4', 'page-count': '8'}
{'page-count': '2'}
{'page-count': '13'}
{'page-count': '8'}
{'page-count': '10'}
{'page-count': '5'}
{'page-count': '4'}
{'page-count': '4'}
{'page-count': '12'}
{'page-count': '7'}
{'page-count': '10'}
{'page-count': '11'}
{'page-count': '8'}
{'page-count': '13'}
{'page-count': '14'}
{}
{'page-co